# 프롬프트 엔지니어링 with ChatGPT

In [1]:
# !pip install openai

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('/home/dheum/projects/sw_temp/.env')  # 코드안에 키를 포함시키지 않게 하기 위해 .env 파일로 분리
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Chat Completion

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(      # .chat.completions: 질의에 대한 응답을 .create(): 생성함
    model="gpt-4o",
    messages=[                  # 배열[] 안에 딕셔너리{} 형태로 입력, text 내용이 입력 토큰을 소비함
        {
            "role": "system",   # 시스템 프롬포트
            "content": [        # 시스템 설정값
                {
                    "type": "text",
                    "text": "당신은 아주 친절한 챗봇입니다."
                }
            ]
        },
        {
            "role": "user",     # 유저 프롬포트
            "content": [        # 입력할 메세지 내용
                {
                    "type": "text",
                    "text": "화요일 저녁 뭘 먹으면 좋을까?"
                }
            ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=0.5,            # 응답의 다양성 설정, 0~2 사이 값, 기본값: 1 → 너무 높으면 여러 언어를 섞어 사용함
    max_tokens=2048,            # 응답 최대 길이, 기본값: 2048
    top_p=1,
    frequency_penalty=0,        # 중복되는 토큰에 대한 패널티
    presence_penalty=0          # 토큰 재사용에 대한 패널티
)

In [ ]:
print(response.choices[0].message.content)

화요일 저녁에는 어떤 음식을 드시고 싶은지에 따라 다르겠지만, 몇 가지 추천해드릴게요:

1. **한식**: 김치찌개나 된장찌개 같은 따뜻한 국물 요리와 함께 밥을 드시면 좋을 것 같아요.

2. **양식**: 파스타나 피자를 선택해 보세요. 샐러드를 곁들여도 좋습니다.

3. **중식**: 짜장면이나 탕수육 같은 메뉴도 맛있죠.

4. **일식**: 초밥이나 우동을 드셔보세요. 가볍게 덮밥도 좋습니다.

5. **간단한 식사**: 샌드위치나 샐러드를 준비해서 가볍게 드시는 것도 좋은 선택입니다.

기분에 맞는 음식을 선택해서 맛있는 저녁 식사 하세요!


### 기사 제목 교정

In [ ]:
from openai import OpenAI

def correct_title(query, temperature=0.3):
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # gpt가 인지할 때 하나의 문자열로써 다르게 인지하게 하기 위해 # 추가
    system_instruction = """
당신은 편집장입니다. 기자들이 송고한 기사 제목을 교정하세요.

### 지시사항 ###
- 기사의 제목이 명학하고 주제와 잘 맞도록 수정할 것
- 독자의 관심을 끌 수 있도록 간결하고 임팩트 있는 표현을 사용할 것
- 비속어, 은어 등은 제거하고 의미가 유지되도록 제목을 교정할 것

### 출력 형식 ###
- 기존 제목: [기사의 원래 제목]
- 교정 제목:
    [기사의 교정된 제목]
    [기사의 교정된 제목]
    [기사의 교정된 제목]

### 예시 ###
- 기존 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
- 교정 제목:
    "서울 대형 화제, 수백명 대피!"
    "서울서 화재 수백명 대피해"
    "수백명을 대피하게한 서울에 난 불"
"""
    user_message = f"""
다음 제목을 교정해 주세요.
제목: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,    
        presence_penalty=0      
    )
    return response.choices[0].message.content

In [ ]:
# 출력 형식 1개로 보냈을 때
# - 기존 제목: [기사의 원래 제목]
# - 교정 제목: [기사의 교정된 제목]

print(correct_title("컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요."))

- 기존 제목: 컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요.
- 교정 제목: 컬투쇼 라디오, 주목할 만한 사연 공개!


In [ ]:
# 출력 형식 3개로 보냈을 때
# - 기존 제목: [기사의 원래 제목]
# - 교정 제목:
#     [기사의 교정된 제목]
#     [기사의 교정된 제목]
#     [기사의 교정된 제목]

print(correct_title("컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요."))

- 기존 제목: 컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요.
- 교정 제목:
    "컬투쇼, 기대되는 사연 공개!"
    "컬투쇼, 주목할 만한 사연 소개!"
    "컬투쇼, 화제의 사연을 만나보세요!"


In [ ]:
# 예시 양식을 다르게 제공했을 때
# - 기존 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
# - 교정 제목:
#     "서울 대형 화제, 수백명 대피!"
#     "서울서 화재 수백명 대피해"
#     "수백명을 대피하게한 서울에 난 불"

print(correct_title("컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요."))

- 기존 제목: 컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요.
- 교정 제목:
    "컬투쇼 라디오, 화제의 사연 주목!"
    "컬투쇼 라디오, 대박 사연 기대하세요!"
    "컬투쇼, 주목할 만한 사연 방송 예정!"


### 영단어장 생성 (json 형식으로 저장)
- 팝송 가사에서 단어를 추출하고, 뜻과 다양한 예문 생성

In [12]:
from openai import OpenAI
import json

def extract_eng_words(query, temperature=0.3):
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # gpt가 인지할 때 하나의 문자열로써 다르게 인지하게 하기 위해 # 추가
    system_instruction = """
당신은 영어 팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

### 처리 단계 ###
1. 노래 가사에서 영어 단어 5개를 랜덤하게 추출한다.
2. 추출한 각 단어의 의미를 한글로 나타낸다.
3. 추출한 각 단어별로 유사한 영어 단어를 함께 소개한다.
4. 소개한 유사어에 대한 예문을 작성한다.

### 출력 형식 ###
json 형식으로 출력
- 첫 번째 객체는 json_list여야 한다.
- list 하위에 json_object가 연속된다.

### 출력 예시 ###
[[
    {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어":[
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }},
    {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어":[
        "hi": ["Hi, I'm Emma", <다음 예시>],
        "good morning": ["Good morning, Sam", <다음 예시>]
        ]
    }},
    ... **총 5번 반복**
]]
"""
    user_message = f"""
노래 가사: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "json_object"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,    
        presence_penalty=0      
    )
    return json.loads(response.choices[0].message.content)

In [ ]:
# 영어 가사 (query) → 랜덤하게 5개의 단어 추출 → 단어 뜻(한글), 유사어, 유사어의 예문
result = extract_eng_words("""
Popular, you're gonna be popular
I'll teach you the proper poise when you talk to boys
Little ways to flirt and flounce, ooh!
I'll show you what shoes to wear, how to fix your hair
Everything that really counts to be
""")

In [15]:
result

{'json_list': [{'단어': 'popular',
   '뜻': '인기 있는',
   '유사어': {'famous': ['She became famous after the movie',
     'The famous singer arrived at the event'],
    'well-known': ['He is well-known in the community',
     'The restaurant is well-known for its pasta']}},
  {'단어': 'teach',
   '뜻': '가르치다',
   '유사어': {'instruct': ['The coach will instruct the team',
     'She instructed the students on the project'],
    'educate': ['The program aims to educate young people',
     'He was educated at a private school']}},
  {'단어': 'proper',
   '뜻': '적절한',
   '유사어': {'appropriate': ['Wear appropriate clothing for the interview',
     'The response was appropriate for the situation'],
    'suitable': ['This book is suitable for children',
     'The movie is suitable for all ages']}},
  {'단어': 'flirt',
   '뜻': '추파를 던지다',
   '유사어': {'tease': ['He likes to tease his friends',
     'She teased him about his new haircut'],
    'woo': ['He tried to woo her with flowers',
     'The company is trying to

In [17]:
for word_dict in result['json_list']:
    print(word_dict["단어"])
    print(word_dict["뜻"])

    for sim_word in word_dict["유사어"]:
        print(sim_word, ":", word_dict["유사어"][sim_word])
        
    print()

popular
인기 있는
famous : ['She became famous after the movie', 'The famous singer arrived at the event']
well-known : ['He is well-known in the community', 'The restaurant is well-known for its pasta']

teach
가르치다
instruct : ['The coach will instruct the team', 'She instructed the students on the project']
educate : ['The program aims to educate young people', 'He was educated at a private school']

proper
적절한
appropriate : ['Wear appropriate clothing for the interview', 'The response was appropriate for the situation']
suitable : ['This book is suitable for children', 'The movie is suitable for all ages']

flirt
추파를 던지다
tease : ['He likes to tease his friends', 'She teased him about his new haircut']
woo : ['He tried to woo her with flowers', 'The company is trying to woo new customers']

wear
입다
dress : ['She dressed quickly for the party', 'He dressed in a suit for the meeting']
put on : ["Put on your coat, it's cold outside", 'She put on some makeup before leaving']



### ReAct 기법
- Reason & Action 기법 : 현재 상황에 대한 통찰 이후 다음 행동에 대한 작성을 유도하는 기법

##### 냉장고를 부탁해
- query : 냉장고에 있는 재료 목록
- return : query로 만든 재료를 활용한 레시피 작성
- [hint] 현재 상황에 대한 분석/파악 + 행동 계획 → 실행

In [ ]:
from openai import OpenAI

# 내가 한 실습
def finish_my_fridge(query, temperature=0.3):
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
당신은 한식, 양식, 일식, 중식에 모두 능통한 요리사입니다.
주어진 음식 재료를 활용한 요리의 레서피를 작성하세요.

### 지시사항 ###
- 요리의 이름, 필요한 재료, 조리 과정을 작성하시오.
- 입력된 재료 외에도, 필요한 재료가 있다면 필요한 재료에 작성하시오.
- 요리의 이름은 재치있는 표현으로 작명하시오.
- 요리의 예상 조리 시간은 15분입니다.
- 간단하게 조리할 수 있으나, 보기에 먹음직스러운 요리를 선택할 것.

### 출력 형식 ###
- 요리 이름: [요리 이름]
- 사용 재료:
    [필요한 재료 1], [필요한 재료 2], [필요한 재료 3], ... **필요한 모든 재료 나열**
- 조리 과정:
    1. 조리과정 1
    2. 조리과정 2
    3. 조리과정 3
    ... **조리과정 나열**

### 출력 예시 ###
- 요리 이름: 갸루상 케이크
- 사용 재료:
    또띠아 1장, 달걀 2알, 설탕 반 컵, 우유 반컵, 바닐라 아이스크림 2스쿱, 밀가루 적당량, 딸기 4개
- 조리 과정:
    1. 달걀의 흰자와 노른자를 분리한다.
    2. 노른자 2개와 설탕, 우유, 바닐라 아이스크림, 밀가루를 넣어 섞고 커스터드 크림이 될 때까지 전자렌지에 3분 간격으로 돌린다.
    3. 또띠아를 6등분 해 기름에 튀겨낸다.
    4. 튀긴 또띠아 위에 커스터드 크림을 층층이 올리고 반으로 자른 딸기를 토핑한다.
"""
    user_message = f"""
재료 목록: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,    
        presence_penalty=0      
    )
    return response.choices[0].message.content

In [ ]:
print(finish_my_fridge("참치, 식빵, 토마토, 인스턴트 커피"))

- 요리 이름: 참치 토마토 브루스케타 위드 커피 킥

- 사용 재료:
    참치캔 1개, 식빵 2장, 토마토 1개, 인스턴트 커피 1작은술, 올리브 오일 1큰술, 마늘 1쪽, 소금 약간, 후추 약간, 바질 잎 몇 장 (선택 사항)

- 조리 과정:
    1. 식빵을 반으로 잘라 총 4조각을 만든 후, 올리브 오일을 살짝 발라준다.
    2. 마늘을 반으로 잘라 식빵의 한 면에 문질러 향을 입힌다.
    3. 팬을 중불로 달군 후, 식빵을 노릇노릇하게 구워준다.
    4. 토마토를 작은 큐브 모양으로 썰고, 참치는 기름을 제거한 후 잘게 부순다.
    5. 작은 볼에 토마토와 참치를 넣고, 인스턴트 커피, 소금, 후추를 넣어 잘 섞어준다.
    6. 구운 식빵 위에 토마토와 참치 혼합물을 얹는다.
    7. 바질 잎을 얹어 장식하면 완성된다. 

이 요리는 참치와 토마토의 신선한 맛에 커피의 독특한 풍미가 더해져 색다른 맛을 제공합니다.


In [ ]:
from openai import OpenAI

# 강사님 방법
def finish_my_fridge(query, temperature=0.3):
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
너는 냉장고에 있는 재료를 활용해 창의적이고 실용적인 저녁 식사 아이디어를 제안하는 요리 전문가이다.
네 역할은 사용자로부터 받은 재료 목록을 분석하고, 이를 활용할 요리 아이디어를 구상해서,
조리방법을 단계별로 상세히 설명하는 것이다.

### 상황 분석 ###
- 현재 가진 재료는 [사용자가 입력한 재료]입니다.
- [사용자가 입력한 재료]에 적합한 요리 종류를 생각합니다.

### 행동 계획 ###
- 가장 쉽게 만들 수 있는 요리를 제안합니다.
- 각 요리에 필요한 재료와 조리 도구를 확인합니다.
- 부족한 재료가 있다면, 대체 가능한 옵션을 제시합니다.
- 조리 과정을 단게별로 상세히 설명합니다.
- 맛을 더 향상시키기 위한 팁과 주의사항을 함께 제공합니다.

### 실행 ###
- 상세 레시피를 단계별로 작성합니다.
"""
    user_message = f"""
내 냉장고에는 현재 {query}가 있습니다.
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,    
        presence_penalty=0      
    )
    return response.choices[0].message.content

In [ ]:
print(finish_my_fridge("참치, 식빵, 토마토, 인스턴트 커피"))

좋습니다! 당신의 냉장고에 있는 재료로 간단하고 맛있는 "참치 토마토 샌드위치"를 만들어 보겠습니다. 이 요리는 아침이나 점심, 심지어 간단한 저녁 식사로도 적합합니다.

### 필요한 재료 ###
- 참치 (통조림 또는 신선한 것)
- 식빵
- 토마토
- (선택 사항) 마요네즈, 소금, 후추

### 조리 도구 ###
- 칼
- 도마
- 작은 그릇
- 포크
- 토스터 (선택 사항)

### 조리 방법 ###

1. **참치 준비하기**
   - 참치 통조림을 사용한다면, 먼저 물기를 빼고 작은 그릇에 담습니다.
   - 포크를 사용하여 참치를 부드럽게 으깹니다.
   - (선택 사항) 마요네즈 한 스푼을 추가하여 잘 섞어줍니다. 이렇게 하면 참치가 더 부드럽고 크리미해집니다.
   - 소금과 후추로 간을 맞춥니다.

2. **토마토 준비하기**
   - 토마토를 깨끗이 씻은 후, 얇게 슬라이스합니다.

3. **식빵 준비하기**
   - 식빵을 토스터에 구워 바삭하게 만듭니다. (선택 사항)
   - 구운 식빵은 샌드위치의 식감을 더 좋게 만들어 줍니다.

4. **샌드위치 만들기**
   - 식빵 한 쪽에 준비한 참치 믹스를 얹습니다.
   - 그 위에 토마토 슬라이스를 올립니다.
   - 다른 식빵 한 쪽을 덮어 샌드위치를 완성합니다.

5. **서빙하기**
   - 샌드위치를 반으로 잘라 접시에 담아 서빙합니다.
   - 인스턴트 커피를 곁들여 간단한 식사를 완성합니다.

### 추가 팁 ###
- 샌드위치에 신선한 채소(예: 양상추, 오이)를 추가하면 더 풍성한 맛을 즐길 수 있습니다.
- 참치 대신 닭가슴살이나 다른 통조림 생선을 사용할 수도 있습니다.
- 샌드위치에 약간의 머스타드 소스를 추가하면 색다른 풍미를 느낄 수 있습니다.

이렇게 하면 냉장고에 있는 재료로 간단하고 맛있는 샌드위치를 만들 수 있습니다. 즐거운 식사 시간 되세요!


### 면접 질문 생성

In [28]:
from openai import OpenAI

def job_interview(query, temperature=0.3):
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
당신은 토목, 건축 분야에서 박사학위를 취득하였으며, 해당 분야의 면접 전문가입니다.
사용자가 제공한 구인 공고에 근거하여 핵심 질문들로 지원자를 평가할 수 있어야 합니다.
"""
    user_message = f"""
아래 채용 공고의 직무에 대한 예상 질문과 모범 답안을 작성해 주세요.

--- 가이드 ---
인성면접(태도, 성격 등)과 기술면접으로 2개의 섹션으로 나눠서 작성해 주세요.
각 섹션별로 3개의 질문과 답변을 준비해 주세요.

--- 출력형식 ---
# 1. 인성면접
질문1)
답변1)

질문2)
답변2)

질문3)
답변3)

# 2. 기술면접
질문1)
답변1)

질문2)
답변2)

질문3)
답변3)

--- 채용 공고 ---
{query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,    
        presence_penalty=0      
    )
    return response.choices[0].message.content

In [29]:
query="""
<연수 개요>
차세대 원전 구조물을 위한 신기술 개발에 대한 연수를 목적으로 함
<연수의 필요성 및 목적>
원전 구조물 분야 연구는 정부의 탈원전 정책으로 인하여 연구자들의 이탈이 많았던 분야로서 최근 원자력 활용으로 정책이 변경됨에 따라 많은 연구인력이 필요한 분야이며, 특히 젊은 연구자들의 참여가 필요함
<연수 추진일정>
1년차 - 원전 구조물 수명관리 시스템(SLMS)의 분석 및 개선사항 도출, 원전 시공환경과 노출환경을 고려한 실험환경 구축 및 재료 특성 분석
2년차 - 원전 구조물 노후도 평가 관리 체계 개발, 품질관리 가이드라인 마련을 위한 기반기술 개발
3년차 - 원전 차폐 콘크리트의 성능 확보 기술 개발, 노출환경 모사 시공품질 평가기술 개발
<연수 세부내용>
• 연평균 손상률을 활용한 원전 구조물의 미래손상 예측 체계 구축
• 차폐 콘크리트의 물성 평가 및 개선 방안 도출
• 공극 채움성능 비파괴적 계측 방법 도출
• 시공전 배합상태 평가 시험방법(안) 도출
• 온도조건에 따른 콘크리트 수화열 분석
• 재료물성 시험방법(안) 검증 평가
<기대성과>
• 차세대 원전 구조물 전문가로서 기반 확보
• 원전 콘크리트 구조물 점검 절차서 개정(안) 개발 1건
• 재료물성 평가 시험방법(안) 1건 개발
• 시공전 배합상태 평가 시험방법(안) 1건 개발
<연수 후 활용계획>
• 연수활동을 성실히 수행할 경우,
 - 과제기반 테뉴어 방식으로 전환하여 추가적인 연수 지원
 - 연수목적과 부합하는 우리 연구원 연구과제에 외부 전문가로서 참여하여 연수성과를 활용할 수 있도록 지원
"""

print(job_interview(query, temperature=1))

# 1. 인성면접

질문1) 
탈원전 정책 가운데, 원전 구조물 연구를 계속하고자 하는 동기는 무엇입니까?

답변1) 
원전 구조물 분야는 공학적 난제에 대한 도전과 창의적인 해결책을 모색할 수 있는 무궁한 가능성이 있는 분야입니다. 특히 최근 정책 변화에 따라 원전이 다시 주목받고 있는 만큼, 이 시점이 연구자로서 기여할 수 있는 좋은 기회라고 생각합니다. 기술적 난관을 극복하는 과정에서 얻는 보람은 물론, 원전의 안전성을 높여 국민과 환경을 보호할 수 있다는 점에서 큰 책임감을 느끼고 있습니다. 또한, 젊은 연구자로서 새로운 신기술 개발을 통해 분야의 발전에 기여하고 싶습니다.

질문2) 
본 프로젝트에서 가장 큰 도전 과제는 무엇이라고 생각하며, 그런 상황에서 어떻게 대처할 것입니까?

답변2) 
가장 큰 도전 과제는 원전 구조물의 수명관리 및 예측 체계를 효율적으로 구축하는 것이라 생각합니다. 이러한 문제들은 복잡한 변수들로 이루어져 있어, 모든 상황을 완벽히 예측하기 어렵습니다. 저는 체계적인 접근 방식과 협업을 통해 문제를 단계적으로 해결해 나갈 것입니다. 데이터를 기반으로 한 분석력, 실험 환경에서의 철저한 테스트, 그리고 전문가들과의 긴밀한 협력을 통해 도전적인 과제를 해결할 계획입니다.

질문3) 
프로젝트가 성공적으로 진행되기 위해 중요한 팀의 가치는 무엇이라고 생각합니까?

답변3) 
프로젝트가 성공적으로 진행되기 위해 중요한 것은 '소통과 협업'이라고 생각합니다. 원전 구조물 연구와 개발은 각기 다른 전문성을 가진 여러 분야의 연구자들이 모여 공동의 목표를 향해 나아가는 과정이므로, 명확한 의사소통과 역할 분담이 필수적입니다. 또한, 연구 과정에서 각자의 의견을 존중하며 다양한 접근법을 시험해 보는 유연성이 필요하며, 이를 통해 더 혁신적인 해결책을 도출할 수 있다고 생각합니다.

# 2. 기술면접

질문1) 
원전 구조물 수명관리 시스템(SLMS)의 분석 및 개선을 위해 고려해야 할 주요 요소는 무엇입니까?

답변1) 
SLMS에서 가장